In [31]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from string import punctuation as en_punctuation
import re
from collections import Counter
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import issparse
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [112]:
dc['uy'].append(0)

In [154]:
dc[np.nan].append(9)

In [155]:
dc

defaultdict(list, {'uy': [0], nan: [9]})

In [32]:
def reverse_str(strs):
    try:
        idx=strs.index(' ')
    except ValueError:
        idx=-1
    if idx==-1:
        return(strs)
    else:
        return reverse_str(strs[idx+1:len(strs)])+' '+strs[0:idx]

In [33]:
current_dir = r'D:\yuwei\study\competition\Predict-demand-online-classified-ad'

periods_train_dir = os.path.join(current_dir,'data/periods_train.csv')
periods_test_dir = os.path.join(current_dir,'data/periods_test.csv')
train_active_dir = os.path.join(current_dir,'data/train_active.csv')
test_active_dir = os.path.join(current_dir,'data/test_active.csv')
train_dir = os.path.join(current_dir,'data/train.csv')
test_dir = os.path.join(current_dir,'data/test.csv')

In [168]:
periods_train = pd.read_csv(periods_train_dir)

In [34]:
train=pd.read_csv(train_dir)

In [52]:
test=pd.read_csv(test_dir)

In [35]:
train.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [137]:
periods_train.isnull().sum()

item_id               0
activation_date    5790
date_from             0
date_to               0
dtype: int64

In [125]:
periods_train.head(3)

,item_id,date_from,date_to
0,8f5caef7afb0,2017-03-15,2017-03-16
1,66218ff526d1,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-15,2017-03-28


In [127]:
(periods_train.loc[0,'date_to']-periods_train.loc[0,'date_from']).days+1

2

In [171]:
periods_train.head(3)

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28


In [156]:
pd.to_datetime('2017-09-12')

Timestamp('2017-09-12 00:00:00')

In [30]:
agg_cat=['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name','activation_date']

#### (1)按['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name']统计广告价格的均值、中位数、方差、最大最小值等
#### (2)根据训练（测试）数据中出现的激活日期，统计每个激活日期当天在展示的广告数量，广告价格（均值，中位数等）
#### (3)根据训练（测试）数据中出现的激活日期，在每个激活日期当天按['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name']统计在展示的广告数量，广告价格（均值，中位数等）
#### (4)根据训练（测试）数据中出现的激活日期，在每个激活日期当天按[('city','category_name'),('region','category_name'),('city','parent_category_name'),('region','parent_category_name')]统计在展示的广告数量，广告价格（均值，中位数等）
#### (5)按['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name']统计广告展示时长的均值、中位数、方差、最大、最小值
#### (6)按[('city','category_name'),('region','category_name'),('city','parent_category_name'),('region','parent_category_name')]统计广告展示时长的均值、中位数、方差、最大、最小值

In [160]:
a=defaultdict(defaultdict)

In [161]:
a['2019-09-02']=defaultdict(int)

In [164]:
a['2019-09-02']['col']+=1

In [166]:
a['2019-09-02']['col']

1

In [172]:
periods_train.head(1)

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16


In [176]:
periods_train[['date_from','date_to']].iloc[0]['date_from']

'2017-03-15'

In [ ]:
def trans_date(date_row,date_list):
    result_date=''
    for dt in date_list:
        if (date_row['date_from'] <= dt) and (date_row['date_to']>=dt):
            result_date = dt
    date_row['date_from']

In [177]:
a=defaultdict(defaultdict)

In [179]:
a['ere']=defaultdict(int)
a['ere']['ty']+=0

In [184]:
sta_cat_feat=['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name']

In [183]:
ac_dates=train['activation_date'].unique()

In [194]:
def statistic_features(active_file,periods_file,activation_dats,cat_feature,chunk_size=100000):
    # 建立字典，用于按天统计各个特征类别下的广告价格
    sta_dict1=dict()
    for feat in cat_feature:
        sta_dict1[feat] = defaultdict(defaultdict)
        for dt in activation_dats:
            sta_dict1[feat][dt] = defaultdict(list)
        
    print('>>>Loading periods data...')
    perd = pd.read_csv(periods_file)
    perd = perd.loc[perd['activation_date'].notnull()]
    ck=iter(pd.read_csv(active_file,iterator = True,chunksize=chunk_size))
    while True:
        try:
            ck_df = next(ck)
            print(ck_df)
            ck_df = pd.merge(ck_df, perd, on=['item_id', 'activation_date'])
            print(ck_df)
            for idx, data_row in ck_df.iterrows():
                print('-----------------------')
                print(data_row)
                for dt in activation_dats:
                    if (data_row['date_from'] <= dt) and (data_row['date_to']>=dt):
                        print(dt)
                        for feat in cat_feature:
                            print(feat,':',data_row['price'])
                            sta_dict1[feat][dt][data_row[feat]].append(data_row['price'])  
        except StopIteration:
            break
    return sta_dict1

In [195]:
sta_dict=statistic_features(train_dir,periods_train_dir,ac_dates,sta_cat_feat)

>>>Loading periods data...
        item_id       user_id                region          city  \
0  b912c3c6a6ad  e00f8ff2eaf9  Свердловская область  Екатеринбург   
1  2dac0150717d  39aeb48f0017     Самарская область        Самара   

  parent_category_name               category_name                    param_1  \
0          Личные вещи  Товары для детей и игрушки  Постельные принадлежности   
1      Для дома и дачи           Мебель и интерьер                     Другое   

   param_2  param_3                  title  \
0      NaN      NaN  Кокоби(кокон для сна)   
1      NaN      NaN      Стойка для Одежды   

                                         description   price  item_seq_number  \
0  Кокон для сна малыша,пользовались меньше месяц...   400.0                2   
1          Стойка для одежды, под вешалки. С бутика.  3000.0               19   

  activation_date user_type  \
0      2017-03-28   Private   
1      2017-03-26   Private   

                                            

KeyboardInterrupt: 

In [190]:
sta_dict['region']['2017-03-22']

defaultdict(list, {})

In [80]:
ab=train.groupby(['city','parent_category_name'],as_index=False)['item_id'].count().sort_values(by='item_id',ascending=False)

In [97]:
ac=[1,3,5]
ac.append(np.nan)

In [105]:
np.mean([1,3,5])

3.0

In [102]:
np.mean(ac)

nan

In [114]:
dt=pd.to_datetime(train['activation_date'])

In [83]:
(ab['item_id']<10).sum()

6190

In [57]:
train['param_1'].value_counts()

Женская одежда                   226289
Для девочек                      151979
Для мальчиков                    117772
Продам                           113764
С пробегом                        61024
Аксессуары                        38568
Мужская одежда                    35716
Другое                            27874
Игрушки                           26442
Детские коляски                   26248
Сдам                              23535
Ремонт, строительство             21444
Стройматериалы                    20197
iPhone                            18381
Кровати, диваны и кресла          17419
Инструменты                       15904
Для кухни                         15721
Комплектующие                     15085
Детская мебель                    14251
Шкафы и комоды                    11417
Приборы и аксессуары              11200
Для дома                          10064
Транспорт, перевозки               9689
Товары для кормления               8543
Samsung                            8330


In [55]:
periods_train['date_to'].min(),periods_train['date_to'].max()

('2017-03-15', '2017-03-28')

In [46]:
periods_train['date_from'].min(),periods_train['date_from'].max()

('2017-03-15', '2017-03-28')

In [48]:
train['activation_date'].min(),train['activation_date'].max()

('2017-03-15', '2017-04-07')

In [53]:
test['activation_date'].min(),test['activation_date'].max()

('2017-04-12', '2017-04-20')

In [5]:
train.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [ ]:
# 针对训练集和测试集中每一个item_id,统计其激活日期当天同类别下正在展示的广告数量和均价，及该广告价格与均价的比值

In [ ]:
def stat_item_num_and_price(active_file):
    seed_data=pd.read_csv(seed_file)
    period_data=pd.read_csv(period_file)
    

In [56]:
train['activation_date'].value_counts()

2017-03-20    115190
2017-03-27    114863
2017-03-19    114416
2017-03-26    113513
2017-03-28    112885
2017-03-21    110535
2017-03-22    109813
2017-03-15    108615
2017-03-23    106544
2017-03-16    106168
2017-03-17     98773
2017-03-18     97554
2017-03-24     97351
2017-03-25     97104
2017-03-29        87
2017-03-30         3
2017-04-01         3
2017-04-02         3
2017-04-03         2
2017-04-07         1
2017-03-31         1
Name: activation_date, dtype: int64

In [31]:
train[agg_cat].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 8 columns):
city                    1503424 non-null object
param_1                 1441848 non-null object
image_top_1             1390836 non-null float64
param_3                 640859 non-null object
param_2                 848882 non-null object
category_name           1503424 non-null object
region                  1503424 non-null object
parent_category_name    1503424 non-null object
dtypes: float64(1), object(7)
memory usage: 91.8+ MB


In [32]:
train['title'].value_counts()

Платье                                                15550
Туфли                                                  6334
Куртка                                                 6331
Пальто                                                 5251
Джинсы                                                 4758
Комбинезон                                             4506
Кроссовки                                              3196
Костюм                                                 2916
Ботинки                                                2876
Босоножки                                              2760
Юбка                                                   2722
Кофта                                                  2721
Ветровка                                               2432
Сниму 1-к квартиру                                     2379
Сапоги                                                 2191
Свадебное платье                                       2183
Рубашка                                 

In [7]:
periods_train.head(3)

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28
